In [1]:
DEBUG = False
N_DEBUG = 100

In [2]:
import pandas as pd
import time
import numpy as np

from gpt2outputdataset.detector_radford import DetectorRadford
from detectgpt.detector_detectgpt import DetectorDetectGPT
from detector_guo import DetectorGuo
from detector_dummy import DetectorDummy
from explainer_wrappers import LIME_Explainer, SHAP_Explainer, Anchor_Explainer

results = []


In [3]:
import transformers
from transformers import pipeline

import torch
import re
import sklearn
import spacy
nlp = spacy.load("en_core_web_sm")

In [4]:
test = pd.read_pickle("./dataset_test.pkl")

test = test[test["author"] == "human_answers"]
print("len(test_human)", len(test))
documents = test["answer"]
gold_labels = test["author"] == "human_answers" # convention: 0: machine, 1: human, see detector.py




#from gpt2outputdataset.detector_radford import DetectorRadford
#from detectgpt.detector_detectgpt import DetectorDetectGPT
from detector_guo import DetectorGuo
detector_classes = [DetectorRadford,DetectorGuo]

from explainer_wrappers import LIME_Explainer, SHAP_Explainer
explainer_classes = [LIME_Explainer,SHAP_Explainer]

len(test_human) 152


In [5]:
DEVICE = "cuda"
pattern = re.compile(r"<extra_id_\d+>")

base_model_name="facebook/opt-350m"
openai_model = False

cache_dir="./.cache"
# mask_model = transformers.AutoModelForSeq2SeqLM.from_pretrained(model, cache_dir=cache_dir).to(DEVICE)
# mask_tokenizer = transformers.AutoTokenizer.from_pretrained(model, model_max_length=mask_model.config.n_positions, cache_dir=cache_dir)#.to(DEVICE)
do_top_k= False
do_top_p= False

In [6]:

base_model = transformers.AutoModelForCausalLM.from_pretrained(base_model_name, cache_dir=cache_dir).to(DEVICE)




base_tokenizer = transformers.AutoTokenizer.from_pretrained(base_model_name, cache_dir=cache_dir, padding_side='left',)


In [7]:
predictions = None

In [8]:
decoded = None

In [9]:
import os

In [10]:
from tqdm import tqdm
import itertools


In [11]:
documents

112    I've heard of handyman type people making a li...
24     No. Securities brokers/dealers in the United S...
600    Hello and Welcome to ‘Ask A Doctor’ service.I ...
874    Use VTIVX. The "Target Retirement 2045" and "T...
528    Predictive analytics encompasses a variety of ...
                             ...                        
728    Hi, dearI have gone through your question. I c...
640    Hi,Dear thanks for your query.this is a chroni...
552    Tensor Processing Unit (TPU) is an AI accelera...
148    I believe that capital gains do affect AGI, bu...
354    I know it may not last longer but i was able t...
Name: answer, Length: 152, dtype: object

# Generate/Load documents
Generate or load documents from `./contrastivity_label_flip_pairs.csv`

In [12]:
columns=["Detector", "Original", "Prompt", "Edited"]

In [13]:
if os.path.isfile("./contrastivity_label_flip_pairs.csv"):
    df = pd.read_csv("./contrastivity_label_flip_pairs.csv")
else: 
    df = pd.DataFrame([], columns=columns)
    # write headers (mode != "a")
    df.to_csv("./contrastivity_label_flip_pairs.csv", encoding="UTF-8", index=False)
df

,Detector,Original,Prompt,Edited
0,DetectorRadford,I've heard of handyman type people making a li...,I've heard of handyman type people making a li...,I've heard of handyman type people making a li...
1,DetectorRadford,No. Securities brokers/dealers in the United S...,No. Securities brokers/dealers in the United S...,No. Securities brokers/dealers in the United S...
2,DetectorRadford,Hello and Welcome to ‘Ask A Doctor’ service.I ...,Hello and Welcome to ‘Ask A Doctor’ service.I ...,Hello and Welcome to ‘Ask A Doctor’ service.I ...
3,DetectorRadford,"Use VTIVX. The ""Target Retirement 2045"" and ""T...","Use VTIVX. The ""Target Retirement 2045"" and ""T...","Use VTIVX. The ""Target Retirement 2045"" and ""T..."
4,DetectorRadford,Predictive analytics encompasses a variety of ...,Predictive analytics encompasses a variety of ...,Predictive analytics encompasses a variety of ...
...,...,...,...,...
362,DetectorDetectGPT,Welcome at HCM I have gone through your que...,Welcome at HCM I have gone through your que...,Welcome at HCM I have gone through your que...
363,DetectorDetectGPT,On what basis did you do your initial allocati...,On what basis did you do your initial allocati...,On what basis did you do your initial allocati...
364,DetectorDetectGPT,"Peter A. Wegner (August 20, 1932 – July 27, 20...","Peter A. Wegner (August 20, 1932 – July 27, 20...","Peter A. Wegner (August 20, 1932 – July 27, 20..."
365,DetectorDetectGPT,"Hi, dearI have gone through your question. I c...","Hi, dearI have gone through your question. I c...","Hi, dearI have gone through your question. I c..."


In [14]:
df.groupby("Detector").count()

,Original,Prompt,Edited
Detector,,,
DetectorDetectGPT,76,76,76
DetectorGuo,149,68,68
DetectorRadford,142,142,142


In [15]:
# for detector_class in detector_classes:
#     detector = detector_class()
#     for document in tqdm(documents[detector.predict_label(documents).astype(bool)], desc="Generating perturbations"): # only use those where f(x) = human
#         np.random.seed(42)
#         torch.manual_seed(42)

#         if df[df["Detector"] == detector.__class__.__name__]["Original"].str.contains(document, regex=False).any(): # check if document is in csv, if yes, skip
#             continue
#         doc = nlp(document)
#         n_tokens_original = len(base_tokenizer(document, return_tensors="pt", padding=True).to(DEVICE).input_ids[0])

#         substrings = [''.join(token.text_with_ws for token in doc[:-i]) for i in range(1,len(document)) ]
#         substrings = [substring for substring in substrings if substring != ""] # at least one token

#         n_generations_per_lenght = 5
#         substrings = list(itertools.chain.from_iterable(itertools.repeat(s, n_generations_per_lenght) for s in substrings))

#         batch_size = 20 # adjust so it fits in your GPU memory
#         row = (detector.__class__.__name__, document, None, None) # to mark the document in the csv cache if no pair is found
#         for batch in (sklearn.utils.gen_batches(len(substrings), batch_size)):
#             encoded = base_tokenizer(substrings[batch], return_tensors="pt", padding=True).to(DEVICE)


#             outputs = base_model.generate(**encoded, min_length=n_tokens_original-5, max_length=n_tokens_original+5, do_sample=True, pad_token_id=base_tokenizer.eos_token_id, eos_token_id=base_tokenizer.eos_token_id)
#             decoded = base_tokenizer.batch_decode(outputs, skip_special_tokens=True)

#             predictions = detector.predict_label(decoded)
#             if any(predictions != 1):
#                 first_new_label = (predictions!=1).argmax(axis=0)
#                 assert decoded[first_new_label] != document
#                 #                                   original  prompt                              first instance that flips label
#                 row = (detector.__class__.__name__, document, substrings[batch][first_new_label], decoded[first_new_label])
#                 break
#         pd.DataFrame([row], columns=columns).to_csv("./contrastivity_label_flip_pairs.csv", mode="a", encoding="UTF-8", index=False, header=False)


In [16]:
df = pd.read_csv("./contrastivity_label_flip_pairs.csv")

In [17]:
df = df.dropna()
df

,Detector,Original,Prompt,Edited
0,DetectorRadford,I've heard of handyman type people making a li...,I've heard of handyman type people making a li...,I've heard of handyman type people making a li...
1,DetectorRadford,No. Securities brokers/dealers in the United S...,No. Securities brokers/dealers in the United S...,No. Securities brokers/dealers in the United S...
2,DetectorRadford,Hello and Welcome to ‘Ask A Doctor’ service.I ...,Hello and Welcome to ‘Ask A Doctor’ service.I ...,Hello and Welcome to ‘Ask A Doctor’ service.I ...
3,DetectorRadford,"Use VTIVX. The ""Target Retirement 2045"" and ""T...","Use VTIVX. The ""Target Retirement 2045"" and ""T...","Use VTIVX. The ""Target Retirement 2045"" and ""T..."
4,DetectorRadford,Predictive analytics encompasses a variety of ...,Predictive analytics encompasses a variety of ...,Predictive analytics encompasses a variety of ...
...,...,...,...,...
362,DetectorDetectGPT,Welcome at HCM I have gone through your que...,Welcome at HCM I have gone through your que...,Welcome at HCM I have gone through your que...
363,DetectorDetectGPT,On what basis did you do your initial allocati...,On what basis did you do your initial allocati...,On what basis did you do your initial allocati...
364,DetectorDetectGPT,"Peter A. Wegner (August 20, 1932 – July 27, 20...","Peter A. Wegner (August 20, 1932 – July 27, 20...","Peter A. Wegner (August 20, 1932 – July 27, 20..."
365,DetectorDetectGPT,"Hi, dearI have gone through your question. I c...","Hi, dearI have gone through your question. I c...","Hi, dearI have gone through your question. I c..."


In [18]:
df[df["Original"] == df["Edited"]]

,Detector,Original,Prompt,Edited


In [19]:
from IPython.core.display import HTML

In [20]:
# basic idea: assert that exp(original)[original - prompt] <substantially different than> exp(label_flip_example)[label_flip_example - prompt]
# i.e. the new/changed section is assigned the opposite label (TODO hard coded: "machine") more often

In [21]:
import krippendorff

In [22]:
from sklearn.metrics.pairwise import cosine_similarity

In [23]:
import difflib


In [24]:
results = []
for detector_class in detector_classes:
    detector = detector_class()
    for explainer_class in explainer_classes:
            explainer = explainer_class(detector)
            for idx, (_, original, prompt, edited) in tqdm(df[df["Detector"] == detector.__class__.__name__].iterrows(), desc="Gathering results"):
                
                # if not (explainer.is_cached(original)) or not(explainer.is_cached(edited)):
                #      continue
                # The generation strategy above uses spacy's tokenizer (where punctuation chars are end up in seperate tokens). 
                # The explanation methods have their own tokenizers and FI scores are reported irt to those tokens
                # For LIME, multiple punctuation chars can end up in the same token e.g.: "!)" is one token, "!" too, "#+#++..,++##" as well. 
                # This is problematic when comparing explanations:
                # i.e. this can fail: assert explainer.tokenize(original)[0:lenght_promt] == explainer.tokenize(edited)[0:lenght_promt]
                
                # For this experiment, it is only important to separate the "prompt", which is the common part between the two documents, from the generated/cut parts
                # Determining the bounds of the prompt has to be done AFTER tokenization:
                # Strategy: change bounds until assert explainer.tokenize(original)[0:lenght_promt] == explainer.tokenize(edited)[0:lenght_promt] passes
                lenght_promt = len(explainer.tokenize(prompt)) # this would suffice for SHAP, but not for lime (because it sometimes collapses punctuation chars into one token) 
                while explainer.tokenize(original)[0:lenght_promt] != explainer.tokenize(edited)[0:lenght_promt] or len(explainer.tokenize(original)[lenght_promt:]) == 0 or len(explainer.tokenize(edited)[lenght_promt:]) == 0:    
                    lenght_promt -= 1 
                lenght_promt = max(lenght_promt, 1) # if the first word is followed by a punctuation char, e.g., "Example! Is a sentence." and the prompt is just "Example", the loop above would set lenght_promt=0. This happens as LIME tokenizes this to ['Example!', 'Is', 'a', 'sentence.']
                assert explainer.tokenize(original)[1:lenght_promt] == explainer.tokenize(edited)[1:lenght_promt]
                       
                # get cut/edited parts
                exp_original_cut_part = explainer.get_fi_scores(original, fill=True)[0][lenght_promt:] # TODO hard coded: "machine"
                exp_edited_new_part = explainer.get_fi_scores(edited, fill=True)[0][lenght_promt:] # setting fill=True returns all features (not just the top_k) 
                fi_scores_exp_original_cut_part = np.array([fi_score for _, fi_score in exp_original_cut_part])
                fi_scores_exp_edited_new_part =   np.array([fi_score for _, fi_score in exp_edited_new_part])
                

                # get common part
                exp_original_common_part = explainer.get_fi_scores(original, fill=True)[0][0:lenght_promt]
                exp_edited_common_part = explainer.get_fi_scores(edited, fill=True)[0][0:lenght_promt]
                fi_scores_exp_original_common_part = np.array([fi_score for _, fi_score in exp_original_common_part])
                fi_scores_exp_edited_common_part=    np.array([fi_score for _, fi_score in exp_edited_common_part])
                
                # build result row
                if len(fi_scores_exp_original_cut_part) == 0 or(fi_scores_exp_edited_new_part.shape[0] == 0 or fi_scores_exp_edited_common_part.shape[0] == 0 ):
                  #  print(cannonical_form.shape)
                    print(lenght_promt)
                    print(original)
                    print(prompt)
                    print(edited)
                    print(fi_scores_exp_edited_new_part)
                    print(fi_scores_exp_edited_common_part)
                    print(fi_scores_exp_original_cut_part)
               
                max_fi_in_new_part = fi_scores_exp_edited_new_part.max() > fi_scores_exp_edited_common_part.max()


                cannonical_form = np.vstack([fi_scores_exp_original_common_part, fi_scores_exp_edited_common_part])
                
                assert len(fi_scores_exp_original_cut_part) > 0
                assert len(fi_scores_exp_edited_new_part) > 0
                assert len(fi_scores_exp_original_common_part) > 0
                assert len(fi_scores_exp_edited_common_part) > 0

                k_alpha = None
                if np.all(cannonical_form==0): # the krippendorff library requires items to not all be the same
                    k_alpha = 1
                else:
                    k_alpha = krippendorff.alpha(cannonical_form, level_of_measurement="interval")

                cos_sim = cosine_similarity(fi_scores_exp_original_common_part.reshape(1, -1),fi_scores_exp_edited_common_part.reshape(1, -1))[0,0]
                #columns = ["Explainer",                  "Detector",                 "Mean FI Original Cut Part",               "Mean FI Edited New Part",              "Mean FI Original Common Part",              "Mean FI Edited Common Part",             "cos_sim Common Part",  "Krippendorff's Alpha Common Part", "Maximum FI for Machine in New Part"                                                                                                             ]
                row =     (explainer.__class__.__name__, detector.__class__.__name__, np.mean(fi_scores_exp_original_cut_part), np.mean(fi_scores_exp_edited_new_part), np.mean(fi_scores_exp_original_common_part), np.mean(fi_scores_exp_edited_common_part),  cos_sim,               k_alpha,                             max_fi_in_new_part                                                      )
                results.append(row)



Gathering results: 142it [00:01, 110.33it/s]
Gathering results: 142it [00:05, 25.58it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Gathering results: 68it [00:00, 148.47it/s]
Gathering results: 68it [00:00, 78.16it/s]


In [25]:
columns = ["Explainer",                  "Detector",                 "Mean FI Original Cut Part",               "Mean FI Edited New Part",              "Mean FI Original Common Part",              "Mean FI Edited Common Part",    "cos_sim Common Part", "Krippendorff's Alpha Common Part","Maximum FI for Machine in New Part"]
dff = pd.DataFrame(results, columns=columns)
dff

,Explainer,Detector,Mean FI Original Cut Part,Mean FI Edited New Part,Mean FI Original Common Part,Mean FI Edited Common Part,cos_sim Common Part,Krippendorff's Alpha Common Part,Maximum FI for Machine in New Part
0,LIME_Explainer,DetectorRadford,-0.022479,0.001395,-0.008005,-0.012822,0.640168,0.525773,True
1,LIME_Explainer,DetectorRadford,-0.002890,0.001969,-0.001524,-0.003730,0.000000,-0.035247,True
2,LIME_Explainer,DetectorRadford,-0.021161,0.006397,-0.018898,-0.007048,0.690172,0.677160,False
3,LIME_Explainer,DetectorRadford,0.000000,0.000000,-0.002178,-0.001524,0.138192,0.096623,False
4,LIME_Explainer,DetectorRadford,0.000000,0.000275,-0.000805,0.000485,0.000000,0.002750,False
...,...,...,...,...,...,...,...,...,...
415,SHAP_Explainer,DetectorGuo,0.000003,0.006129,-0.000016,-0.007838,0.983648,-0.798943,True
416,SHAP_Explainer,DetectorGuo,0.000274,0.026362,-0.000253,-0.006295,0.836267,0.005847,True
417,SHAP_Explainer,DetectorGuo,-0.000001,0.005787,-0.000005,-0.002009,0.244874,0.010832,True
418,SHAP_Explainer,DetectorGuo,0.005107,0.014641,-0.010522,-0.015239,0.796072,0.392361,True


In [26]:
results_detector_level = dff.set_index(["Explainer", "Detector"]).groupby(["Explainer", "Detector"]).mean()
results_detector_level

Mean FI Original Cut Part  \
Explainer      Detector                                     
LIME_Explainer DetectorGuo                       0.000003   
               DetectorRadford                  -0.005011   
SHAP_Explainer DetectorGuo                       0.000406   
               DetectorRadford                  -0.011269   

                                Mean FI Edited New Part  \
Explainer      Detector                                   
LIME_Explainer DetectorGuo                     0.000636   
               DetectorRadford                 0.003827   
SHAP_Explainer DetectorGuo                     0.010159   
               DetectorRadford                 0.008824   

                                Mean FI Original Common Part  \
Explainer      Detector                                        
LIME_Explainer DetectorGuo                          0.000004   
               DetectorRadford                     -0.006190   
SHAP_Explainer DetectorGuo                         -0.000436   
               DetectorRadford                     -0.008248   

                                Mean FI Edited Common Part  \
Explainer      Detector                                      
LIME_Explainer DetectorGuo                        0.000510   
               DetectorRadford                   -0.005440   
SHAP_Explainer DetectorGuo                       -0.003422   
               DetectorRadford                   -0.007162   

                                cos_sim Common Part  \
Explainer      Detector                               
LIME_Explainer DetectorGuo                 0.054476   
               DetectorRadford             0.285810   
SHAP_Explainer DetectorGuo                 0.403014   
               DetectorRadford             0.755039   

                                Krippendorff's Alpha Common Part  \
Explainer      Detector                                            
LIME_Explainer DetectorGuo                              0.006498   
               DetectorRadford                          0.242693   
SHAP_Explainer DetectorGuo                             -0.216423   
               DetectorRadford                          0.635299   

                                Maximum FI for Machine in New Part  
Explainer      Detector                                             
LIME_Explainer DetectorGuo                                0.779412  
               DetectorRadford                            0.563380  
SHAP_Explainer DetectorGuo                                0.955882  
               DetectorRadford                            0.542254

In [27]:
results_explainer_level = dff.set_index(["Explainer", "Detector"]).groupby(["Explainer"]).mean()
results_explainer_level

,Mean FI Original Cut Part,Mean FI Edited New Part,Mean FI Original Common Part,Mean FI Edited Common Part,cos_sim Common Part,Krippendorff's Alpha Common Part,Maximum FI for Machine in New Part
Explainer,,,,,,,
LIME_Explainer,-0.003387,0.002794,-0.004184,-0.003513,0.210902,0.166211,0.633333
SHAP_Explainer,-0.007489,0.009256,-0.005718,-0.005951,0.641050,0.359503,0.676190


In [28]:
from scipy.stats.mstats import ttest_rel
from scipy.stats.mstats import ttest_1samp

In [29]:
dff.set_index(["Explainer", "Detector"]).groupby(["Explainer"]).apply(lambda group: ttest_rel(group["Mean FI Original Cut Part"], group["Mean FI Edited New Part"]))

Explainer
LIME_Explainer       (-9.860397927464787, 4.4651274511593e-19)
SHAP_Explainer    (-19.458669358910566, 8.295188895065286e-49)
dtype: object

In [30]:
dff.set_index(["Explainer", "Detector"]).groupby(["Explainer"]).apply(lambda group: ttest_rel(group["Mean FI Original Common Part"], group["Mean FI Edited Common Part"]))

Explainer
LIME_Explainer    (-2.317917477363351, 0.02142119917196867)
SHAP_Explainer     (0.2997543385384911, 0.7646624508180425)
dtype: object

In [31]:
dff.set_index(["Explainer", "Detector"]).groupby(["Explainer"]).apply(lambda group: ttest_1samp(group["Maximum FI for Machine in New Part"], popmean=0.5))

Explainer
LIME_Explainer    (3.9999999999999982, 8.789002398047055e-05)
SHAP_Explainer    (5.443475073719952, 1.4574983832591005e-07)
dtype: object